In [1]:
using Revise
using Pkg
TAMBO_PATH = "/Users/jlazar/research/TAMBO-MC/Tambo/"
Pkg.activate(TAMBO_PATH)

using Tambo
using CSV
using JLD2
using Plots
using PyCall
using Glob
using StaticArrays
include("../paperstyle.jl")

  Activating project at `~/research/TAMBO-MC/Tambo`


no_bg_dark! (generic function with 1 method)

In [2]:
ak = pyimport("awkward")
np = pyimport("numpy")

PyObject <module 'numpy' from '/usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/__init__.py'>

In [3]:
csv = CSV.File("/Users/jlazar/Downloads/CircleTest.csv")
idxs = getindex.(csv, 16)

143-element Vector{Int64}:
   1
   1
   4
   5
   6
   6
   6
   7
   9
   9
  10
  11
  11
   ⋮
  88
  88
  90
  91
  91
  93
  93
  93
  95
  96
  98
 100

In [4]:
const config = SimulationConfig(tambo_coordinates=whitepaper_coord, θmax=π/2)
const geo = Tambo.Geometry(config)
const plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)

const zmin = -1100units.m
const zmax = 1100units.m
const ycorsika = SVector{3}([0.89192975455881607, 0.18563051261662877, -0.41231374670066206])
const xcorsika = SVector{3}([0, -0.91184756344828699, -0.41052895273466672])
const zcorsika = whitepaper_normal_vec.proj
const xyzcorsika = inv([
  xcorsika.x xcorsika.y xcorsika.z;
  ycorsika.x ycorsika.y ycorsika.z;
  zcorsika.x zcorsika.y zcorsika.z;
 ])

3×3 Matrix{Float64}:
  1.38778e-16   0.89193    0.452174
 -0.911848      0.185631  -0.366163
 -0.410529     -0.412314   0.813304

In [6]:
function Base.getindex(events::Vector{Tambo.CorsikaEvent}, s::String)
    return getfield.(events, Symbol(s))
end

In [8]:
function loadcorsika(file::String)
  events = Tambo.CorsikaEvent[]
  x = nothing
  try
    x = ak.from_parquet(file)
  catch
    return events
  end
  if length(x)==0
    return events
  end
  xs = x["x"].to_numpy() .* units.m
  ys = x["y"].to_numpy() .* units.m
  zs = x["z"].to_numpy() .* units.m
  new_poss = []
  for (x, y, z) in zip(xs, ys, zs)
#     push!(new_poss, [x,y,z])
    push!(new_poss, xyzcorsika * [x,y,z])
  end
  xs = getindex.(new_poss, 1)
  ys = getindex.(new_poss, 2)
  zs = getindex.(new_poss, 3)
  ts = x["time"].to_numpy() .* units.second
  ws = x["weight"].to_numpy() .* 1.0
  ids = x["pdg"].to_numpy()
  es = x["kinetic_energy"].to_numpy() .* units.GeV
  for tup in zip(ids, es, xs, ys, zs, ts, ws)
    push!(events, Tambo.CorsikaEvent(tup...))
  end
  return events
end

function loadcorsika(files::Vector{Any})
  println
  events = Tambo.CorsikaEvent[]
  for file in files
    events = [events; loadcorsika(file)]
  end
  return events
end

loadcorsika (generic function with 2 methods)

In [9]:
directories = glob("sim_test_*_1", "/Users/jlazar/research/TAMBO-MC/resources/test_data/circle_test/n/holylfs05/LABS/arguelles_delgado_lab/Lab/common_software/source/corsika8/corsika-work/CircleTest")
run_numbers = Int[]
for dir in directories
    run_number = parse(Int, split(dir, "_")[end-1])
    if run_number in run_numbers
        continue
    end
    push!(run_numbers, run_number)
end

In [10]:
files = []
for run_number in run_numbers
    tmp = []
    
    for x in glob("sim_test_$(run_number)_*", "/Users/jlazar/research/TAMBO-MC/resources/test_data/circle_test/n/holylfs05/LABS/arguelles_delgado_lab/Lab/common_software/source/corsika8/corsika-work/CircleTest")
        push!(tmp, "$(x)/particles/particles.parquet")
    end
    push!(files, tmp)
end

In [12]:
a = []
for (idx, file) in enumerate(files)
    events = loadcorsika(file)
    push!(a, (-length(events), idx))
end
a = sort(a)

72-element Vector{Any}:
 (-5365, 3)
 (-4356, 68)
 (-3568, 47)
 (-2382, 21)
 (-1763, 13)
 (-1507, 35)
 (-1506, 59)
 (-1495, 11)
 (-1453, 19)
 (-1196, 44)
 (-1066, 58)
 (-1034, 52)
 (-769, 20)
 ⋮
 (0, 51)
 (0, 53)
 (0, 56)
 (0, 60)
 (0, 61)
 (0, 62)
 (0, 63)
 (0, 64)
 (0, 65)
 (0, 69)
 (0, 71)
 (0, 72)

In [13]:
for (negnevts, idx) in sort(a)
    if negnevts >= 0
        continue
    end
    file = files[idx]
    runn = parse(Int, split(file[1], "_")[end-1])
    row = first(csv[idxs.==runn])
    events = loadcorsika(file);
    base_plt = plot(size=(500, 500), xlimits=(first(xs), last(xs))./units.km, ylimits=(first(ys), last(ys))./units.km)

    println(sum(mask))

    contour!(
        base_plt,
        xs ./ units.km,
        ys ./ units.km,
        @. (geo(xs', ys) + geo.tambo_offset.z) / units.km;
        fill=true,
        color=palette(:lapaz),
        clims=(1.5, 5)
    )

    scatter!(
        base_plt,
        getfield.(detection_modules, :x) ./ units.km,
        getfield.(detection_modules, :y) ./ units.km,
        alpha=0.5,
        markersize=3,
        color="black",
        markerstrokewidth=0
    )
    time_mask = events["time"] .< 0.0001units.second

    cg = cgrad(:lightrainbow, rev=true)

    thin = 1

    ltmin = minimum(log.(10, events[time_mask][begin:thin:end]["time"]/units.second))
    cs = get.(Ref(cg), (log.(10, events[time_mask][begin:thin:end]["time"]/units.second) .- ltmin) / (-4 - ltmin))
    scatter!(
        base_plt,

        events[time_mask][begin:thin:end]["x"] / units.km,
        events[time_mask][begin:thin:end]["y"] / units.km,
        zcolor=(events[time_mask][begin:thin:end]["z"] .+ geo.tambo_offset.z) ./ units.km,
        color=palette(:lapaz),
        clims=(1.5, 5)
    #     color=cs,
    )
    scatter!(base_plt, [row.inter_x], [row.inter_y], markersize=10)
    savefig("/Users/jlazar/research/TAMBO-MC/figures/circle_test/event_$(runn).pdf")
end

732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
732
